In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-10-16 20:42:12.733445: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-16 20:42:12.739924: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-16 20:42:12.759203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1729122132.792442 1950964 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1729122132.801753 1950964 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 20:42:12.835559: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [1]:
src_path = './classes'
num_classes = 5
import os
import imghdr

# Function to filter out unsupported image formats
def validate_image(file_path):
  """Validates if the given file is a supported image format.
  
  Args:
    file_path: Path to the image file.

  Returns:
    True if the file is a supported image format, False otherwise.
  """
  supported_formats = ["jpeg", "png", "gif", "bmp"]
  try:
    img_type = imghdr.what(file_path)
    return img_type in supported_formats
  except:
    return False

# List all image files in the source directory
image_files = []
for class_name in os.listdir(src_path):
  class_path = os.path.join(src_path, class_name)
  if os.path.isdir(class_path):
    for file_name in os.listdir(class_path):
      file_path = os.path.join(class_path, file_name)
      if os.path.isfile(file_path):
        image_files.append(file_path)

# Filter out unsupported image files
valid_image_files = [file for file in image_files if validate_image(file)]

# Filter out unsupported image files from the directory list
for file_path in image_files:
  if file_path not in valid_image_files:
    print(f"Removing unsupported file: {file_path}")
    os.remove(file_path)
    


Removing unsupported file: ./classes/gol/image1.jpeg
Removing unsupported file: ./classes/gol/image16.jpeg


/tmp/ipykernel_1950964/1312471093.py:4: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [4]:

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    src_path,
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical',
    subset='training',
    validation_split=0.2,
    seed=123,
)

# Resize layer (if further resizing or additional preprocessing is needed)
resize_layer = tf.keras.Sequential([
    tf.keras.layers.Resizing(224, 224)
])

# Apply resizing and other augmentations (if any) to the dataset
train_data = train_data.map(lambda x, y: (resize_layer(x), y))

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    src_path,
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical',
    subset='validation',
    validation_split=0.2,
    seed=123
)

val_data = val_data.map(lambda x, y: (resize_layer(x), y))

Found 702 files belonging to 5 classes.
Using 562 files for training.
Found 702 files belonging to 5 classes.
Using 140 files for validation.


2024-10-16 20:42:22.565152: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)

In [6]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [7]:

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [8]:
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [10]:
model.fit(train_data, epochs=10, validation_data=val_data)

Epoch 1/10
10/18 ━━━━━━━━━━━━━━━━━━━━ 40s 5s/step - accuracy: 0.3167 - loss: 2.2350

KeyboardInterrupt: 